In [1]:
from sqlalchemy_function import FunctionMixin, FunctionRelator

# standard session creation
from sqlalchemy import create_engine, Column, ForeignKey, Integer, String
from sqlalchemy.orm import relationship, sessionmaker, scoped_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.orderinglist import ordering_list

engine = create_engine('sqlite:///:memory:')
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)
session = Session()
Base = declarative_base()

# define a Function model parent
class Parent(FunctionRelator, Base):
    __tablename__ = 'parent'
    id = Column(Integer, primary_key=True)

    # Fuction models must reference their parent with a `parent` attribute
    functions = relationship('Function', backref='parent')

# define a Function model with the FunctionMixin
class Function(FunctionMixin, Base):
    __tablename__ = 'function'
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey('parent.id'))

Base.metadata.create_all(engine)

In [2]:
def foo(parent, *args, **kwargs):
    print('My parent is', parent)
    print('My args are', args)
    print('My kwargs are', kwargs)
    return 'return value'

In [3]:
parent = Parent()
session.add(parent)
session.commit()
function = Function(
    parent, func=foo, args=['hello world'], kwargs={'goodbye': 'moon'}
)
parent.functions[0]()

My parent is <__main__.Parent object at 0x7f8b4e7c9438>
My args are ('hello world',)
My kwargs are {'goodbye': 'moon'}


'return value'

In [4]:
@Function.register
def foo(parent, *args, **kwargs):
    print('My parent is', parent)
    print('My args are', args)
    print('My kwargs are', kwargs)
    return 'return value'

parent = Parent()
Function.foo(parent, 'hello world', goodbye='moon')
parent.functions[0]()

My parent is <__main__.Parent object at 0x7f8b4d200518>
My args are ('hello world',)
My kwargs are {'goodbye': 'moon'}


'return value'

In [5]:
parent.functions = foo
print(parent.functions)
parent.functions[0]()

My parent is <__main__.Parent object at 0x7f8b4d200518>
My args are ()
My kwargs are {}


'return value'

In [6]:
def bar(parent):
    print('My parent is', parent)
    return 'bar return value'

parent.functions = [foo, bar]
print(parent.functions)
[f() for f in parent.functions]

[<__main__.Function object at 0x7f8b4d2004e0>, <__main__.Function object at 0x7f8b4d2005f8>]
My parent is <__main__.Parent object at 0x7f8b4d200518>
My args are ()
My kwargs are {}
My parent is <__main__.Parent object at 0x7f8b4d200518>


['return value', 'bar return value']